### Assignment 2

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import re
from nltk.stem import PorterStemmer
import nltk
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from gensim.models import Word2Vec



nltk.download('stopwords')
nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kkasp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\kkasp\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


In [16]:
file_path = r'IMDB Dataset.csv'
data = pd.read_csv(file_path)
print(data.head(20))


In [17]:
X = data['review']

def transform_bin(x):
  if x == "positive":
    return 1
  else:
    return 0
data['sentiment'] = data['sentiment'].apply(lambda x : transform_bin(x))

y = data['sentiment']


In [18]:
# Data split (80% training, 20% validation)
train_data, X_test, train_label, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(train_data, train_label, test_size=0.2, random_state=42)

X_train = X_train.to_frame()
X_test = X_test.to_frame()
X_val = X_val.to_frame()

# Display the sizes of the datasets
print("Training Data Size:", len(X_train))
print("Validation Data Size:", len(X_val))
print("Test Data Size:", len(X_test))

Training Data Size: 32000
Validation Data Size: 8000
Test Data Size: 10000


##### Text pre-processing

In [19]:
X_train['review']

11794                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [20]:
def clean_text(text):
    # Remove unusual or non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize the text into words for processing
    words = text.split()

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]


    return filtered_words

X_train['cleaned_review'] = X_train['review'].apply(clean_text)
X_val['cleaned_review']  = X_val['review'].apply(clean_text)
X_test['cleaned_review']  = X_test['review'].apply(clean_text)


# Display the first few cleaned reviews
print("Cleaned Reviews with Stopwords Removed:")
print(X_train['cleaned_review'].head(20))

Cleaned Reviews with Stopwords Removed:
11794                                                                                                                                                                                                                                                           [fault, actors, put, great, performances, overall, story, well, executed, movie, opens, great, zinger, crazy, old, guy, forces, young, Aborigine, girls, car, road, forced, endure, 40, minutes, character, development, entirely, new, group, characters, dont, know, 40, minutes, turns, ones, eventually, discover, girls, body, story, progresses, therebr, br, story, pick, point, really, goes, nowhere, 2, hours, asked, point, see, characters, struggle, accusations, racism, stupidity, handled, discovery, story, ultimately, unsatisfying, felt, unfinished, well, acted, theres, strong, enough, backbone, film, warrant, recommending]
24925                                                                      

Further preprocessing: Features and Embeddings

Features

Here we create: Number of Positive lexicon words, Number of Negative lexicon words, Number of nos, and Number of words. After that the values were standardized

In [25]:
# 1. Lexicon Creation
positive_lexicon = set(opinion_lexicon.positive()) #list of positive words from the NLTK package
negative_lexicon = set(opinion_lexicon.negative()) #list of negative words from the NLTK package

# 2. Defining Functions
def counts_lex(review, type = 'neg'):
    count_neg = 0
    count_pos = 0
    for word in review:
      if word.lower() in positive_lexicon:
        count_pos += 1
      if word.lower() in negative_lexicon:
        count_neg += 1
    if type == 'neg':
      return count_neg
    elif type == 'pos':
      return count_pos

# Ensure reviews are tokenized, we use the review column as 'no' and 'not' gets removed by stopwords
def count_nos(review):
    count = 0
    words = review.split()  # Tokenize if review is a single string
    for word in words:
        if word.lower() == "no" or word.lower() == "not":
            count += 1
    return count

# 3. Feature Extraction for Training, Validation, and Test Sets
X_train['NrPos'] = X_train['cleaned_review'].apply(lambda x : counts_lex(x, type = "pos"))
X_train['NrNeg'] = X_train['cleaned_review'].apply(lambda x : counts_lex(x, type = "neg"))
X_train['NrWords'] = X_train['cleaned_review'].apply(lambda x: len(x))
X_train['Nos'] = X_train['review'].apply(lambda x : count_nos(x))

X_val['NrPos'] = X_val['cleaned_review'].apply(lambda x : counts_lex(x, type = "pos"))
X_val['NrNeg'] = X_val['cleaned_review'].apply(lambda x : counts_lex(x, type = "neg"))
X_val['NrWords'] = X_val['cleaned_review'].apply(lambda x: len(x))
X_val['Nos'] = X_val['review'].apply(lambda x : count_nos(x))

X_test['NrPos'] = X_test['cleaned_review'].apply(lambda x : counts_lex(x, type = "pos"))
X_test['NrNeg'] = X_test['cleaned_review'].apply(lambda x : counts_lex(x, type = "neg"))
X_test['NrWords'] = X_test['cleaned_review'].apply(lambda x: len(x))
X_test['Nos'] = X_test['review'].apply(lambda x : count_nos(x))

# 4. Standardizing Features
cols = ['NrPos', 'NrNeg', 'NrWords', 'Nos']
train_features_to_transform = X_train[cols]
val_features_to_transform = X_val[cols]
test_features_to_transform = X_test[cols]

scaler = StandardScaler()
X_train_std = scaler.fit_transform(train_features_to_transform)  # X is your feature matrix
X_val_std = scaler.transform(val_features_to_transform)
X_test_std = scaler.transform(test_features_to_transform)

X_train[cols] = X_train_std
X_val[cols] = X_val_std
X_test[cols] = X_test_std

# 5. Stemming Words
# Apply stemming
stemmer = PorterStemmer()

X_train['Stem Words'] = X_train['cleaned_review'].apply(lambda x : [stemmer.stem(word.lower()) for word in x])
X_val['Stem Words'] = X_val['cleaned_review'].apply(lambda x : [stemmer.stem(word.lower()) for word in x])
X_test['Stem Words'] = X_test['cleaned_review'].apply(lambda x : [stemmer.stem(word.lower()) for word in x])

# 6. Output for debugging
# Display the first few cleaned reviews
print("Training Data with cleaned reviews with Stemming and New Features:")
print(X_train['cleaned_review'].head(20))

Training Data with cleaned reviews with Stemming and New Features:
11794                                                                                                                                                                                                                                                           [fault, actors, put, great, performances, overall, story, well, executed, movie, opens, great, zinger, crazy, old, guy, forces, young, Aborigine, girls, car, road, forced, endure, 40, minutes, character, development, entirely, new, group, characters, dont, know, 40, minutes, turns, ones, eventually, discover, girls, body, story, progresses, therebr, br, story, pick, point, really, goes, nowhere, 2, hours, asked, point, see, characters, struggle, accusations, racism, stupidity, handled, discovery, story, ultimately, unsatisfying, felt, unfinished, well, acted, theres, strong, enough, backbone, film, warrant, recommending]
24925                                           

In [26]:
X_train

,review,cleaned_review,NrPos,NrNeg,NrWords,Nos,Stem Words
11794,"With no fault to the actors (they all put on great performances), the overall story was not very well executed. The movie opens with a great zinger: a crazy old guy forces a young Aborigine girl's car off the road. But then, we're forced to endure 40 minutes of character development with an entirely new group of characters ... and we don't know why until the 40 minutes are up. It turns out that they are the ones who eventually discover the girl's body ... and the story progresses from there.<br /><br />While the story does pick up at that point, it really goes nowhere. After 2 hours, I asked myself: was there a point to this, or was it just to see the characters struggle with accusations of racism and stupidity of how they handled the discovery? The story was ultimately unsatisfying and felt unfinished. While it is well acted, there's not a strong enough backbone in the film to warrant recommending it.","[fault, actors, put, great, performances, overall, story, well, executed, movie, opens, great, zinger, crazy, old, guy, forces, young, Aborigine, girls, car, road, forced, endure, 40, minutes, character, development, entirely, new, group, characters, dont, know, 40, minutes, turns, ones, eventually, discover, girls, body, story, progresses, therebr, br, story, pick, point, really, goes, nowhere, 2, hours, asked, point, see, characters, struggle, accusations, racism, stupidity, handled, discovery, story, ultimately, unsatisfying, felt, unfinished, well, acted, theres, strong, enough, backbone, film, warrant, recommending]",-0.349058,-0.255423,-0.481820,0.761786,"[fault, actor, put, great, perform, overal, stori, well, execut, movi, open, great, zinger, crazi, old, guy, forc, young, aborigin, girl, car, road, forc, endur, 40, minut, charact, develop, entir, new, group, charact, dont, know, 40, minut, turn, one, eventu, discov, girl, bodi, stori, progress, therebr, br, stori, pick, point, realli, goe, nowher, 2, hour, ask, point, see, charact, struggl, accus, racism, stupid, handl, discoveri, stori, ultim, unsatisfi, felt, unfinish, well, act, there, strong, enough, backbon, film, warrant, recommend]"
24925,"The first thing I thought when I saw this films was: It is not really a film, at least it is not what we imagine spontaneously when we hear the word ""film"". it is entirely symbolic, everything in it has a figurative meaning. So if you are not used to express thing in a symbolic way, you will find it strange, if you are not acquainted with philosophy, religion, spiritual life, you will think it's just a fairy-tale... and even a weird one, chaotic. For me ""The legend of Zu"" is perfectly transparent. And I do like it. It tells us in images the story about the fight between light and darkness, the fight that is as old as humanity, and every one who is in search of the sens in this life is confronted with it. The film is obviously made by Buddhists. I am not a Buddhist. My religion and the vision of the world and human is different. But as far as we are all humans and have the same human nature we necessarily have common experiences and can understand each other. It is a really beautiful film! And I which we had more films like this - films that have a meaning. There are too many empty stories which are good only to make time pass more quickly.","[first, thing, thought, saw, films, really, film, least, imagine, spontaneously, hear, word, film, entirely, symbolic, everything, figurative, meaning, used, express, thing, symbolic, way, find, strange, acquainted, philosophy, religion, spiritual, life, think, fairytale, even, weird, one, chaotic, legend, Zu, perfectly, transparent, like, tells, us, images, story, fight, light, darkness, fight, old, humanity, every, one, search, sens, life, confronted, film, obviously, made, Buddhists, Buddhist, religion, vision, world, human, different, far, humans, human, nature, necessarily, common, experiences, understand, really, b

In [23]:
rows_with_not = X_train[X_train['review'].str.contains(r'\bnot\b', case=False, na=False)]
pd.set_option('display.max_colwidth', None)  # Allows columns to display their full content
pd.set_option('display.width', 200) 
rows_with_not

,review,cleaned_review,NrPos,NrNeg,NrWords,Nos,Stem Words
11794,"With no fault to the actors (they all put on great performances), the overall story was not very well executed. The movie opens with a great zinger: a crazy old guy forces a young Aborigine girl's car off the road. But then, we're forced to endure 40 minutes of character development with an entirely new group of characters ... and we don't know why until the 40 minutes are up. It turns out that they are the ones who eventually discover the girl's body ... and the story progresses from there.<br /><br />While the story does pick up at that point, it really goes nowhere. After 2 hours, I asked myself: was there a point to this, or was it just to see the characters struggle with accusations of racism and stupidity of how they handled the discovery? The story was ultimately unsatisfying and felt unfinished. While it is well acted, there's not a strong enough backbone in the film to warrant recommending it.","[fault, actors, put, great, performances, overall, story, well, executed, movie, opens, great, zinger, crazy, old, guy, forces, young, Aborigine, girls, car, road, forced, endure, 40, minutes, character, development, entirely, new, group, characters, dont, know, 40, minutes, turns, ones, eventually, discover, girls, body, story, progresses, therebr, br, story, pick, point, really, goes, nowhere, 2, hours, asked, point, see, characters, struggle, accusations, racism, stupidity, handled, discovery, story, ultimately, unsatisfying, felt, unfinished, well, acted, theres, strong, enough, backbone, film, warrant, recommending]",-0.349058,-0.255423,-0.481820,0.0,"[fault, actor, put, great, perform, overal, stori, well, execut, movi, open, great, zinger, crazi, old, guy, forc, young, aborigin, girl, car, road, forc, endur, 40, minut, charact, develop, entir, new, group, charact, dont, know, 40, minut, turn, one, eventu, discov, girl, bodi, stori, progress, therebr, br, stori, pick, point, realli, goe, nowher, 2, hour, ask, point, see, charact, struggl, accus, racism, stupid, handl, discoveri, stori, ultim, unsatisfi, felt, unfinish, well, act, there, strong, enough, backbon, film, warrant, recommend]"
24925,"The first thing I thought when I saw this films was: It is not really a film, at least it is not what we imagine spontaneously when we hear the word ""film"". it is entirely symbolic, everything in it has a figurative meaning. So if you are not used to express thing in a symbolic way, you will find it strange, if you are not acquainted with philosophy, religion, spiritual life, you will think it's just a fairy-tale... and even a weird one, chaotic. For me ""The legend of Zu"" is perfectly transparent. And I do like it. It tells us in images the story about the fight between light and darkness, the fight that is as old as humanity, and every one who is in search of the sens in this life is confronted with it. The film is obviously made by Buddhists. I am not a Buddhist. My religion and the vision of the world and human is different. But as far as we are all humans and have the same human nature we necessarily have common experiences and can understand each other. It is a really beautiful film! And I which we had more films like this - films that have a meaning. There are too many empty stories which are good only to make time pass more quickly.","[first, thing, thought, saw, films, really, film, least, imagine, spontaneously, hear, word, film, entirely, symbolic, everything, figurative, meaning, used, express, thing, symbolic, way, find, strange, acquainted, philosophy, religion, spiritual, life, think, fairytale, even, weird, one, chaotic, legend, Zu, perfectly, transparent, like, tells, us, images, story, fight, light, darkness, fight, old, humanity, every, one, search, sens, life, confronted, film, obviously, made, Buddhists, Buddhist, religion, vision, world, human, different, far, humans, human, nature, necessarily, common, experiences, understand, really, beauti

In [27]:
nos_stats = X_train['Nos'].agg(['mean', 'max', 'min'])

# Display the results
print(nos_stats)

mean   -9.769963e-18
max     1.153455e+01
min    -8.541290e-01
Name: Nos, dtype: float64


Making Embeddings

When making the embeddings, we choose values of 4 for window and 1 for sg. 1 was chosen for sg to use SkipGram. The data has 40000 reviews, and even though this is sufficient to train a Word2Vec model, rare or domain-specific sentiment words may not be well captured by a CBOW. The window chosen was 4 because the sentiment is often localized in very few words.

In [28]:
reviews = list(X_train['Stem Words'])
model = Word2Vec(
    sentences=reviews,  # Tokenized data
    vector_size=100,             # Embedding dimensionality
    window=4,                    # Context window size
    min_count=2,                 # Minimum frequency for words to be included
    sg=1,                        # Use CBOW (set to 1 for Skip-Gram)
    workers=4,                   # Number of threads for faster training
    epochs=10                    # Number of passes over the data
)

# Save the model for future use
model.save("word2vec_cbow.model")


In [29]:
model.wv['film']

array([-0.07770786,  0.15269166, -0.00447225,  0.034062  ,  0.325359  ,
       -0.12248843,  0.02910477,  0.4072858 , -0.14709984, -0.12952596,
       -0.25982457, -0.3839486 , -0.05629438,  0.42851132,  0.35073417,
        0.10201956, -0.10908537,  0.33942097, -0.20338179, -0.19975029,
        0.29158154, -0.02966006,  0.32787427, -0.00136138,  0.32530898,
       -0.12784657, -0.11470099,  0.22793236, -0.01679601,  0.07158659,
        0.25591916, -0.17610694,  0.296032  , -0.42777047, -0.02839102,
        0.30722722,  0.10857434, -0.04826575, -0.36882648, -0.07117313,
        0.1052228 , -0.256852  ,  0.09142398,  0.2636197 ,  0.14035228,
       -0.00501746, -0.04419826, -0.37684727,  0.0161701 ,  0.07387528,
        0.21617617, -0.18350986, -0.3036567 , -0.2822288 , -0.12519188,
        0.14377035,  0.20223454, -0.21154587, -0.15528145,  0.27358457,
        0.06026944,  0.02316202, -0.09303118,  0.03925332, -0.2652876 ,
        0.14225906,  0.188505  ,  0.19473477, -0.11783864, -0.02

In [30]:
model.wv.most_similar('film')

[('movi', 0.9021115303039551),
 ('towelhead', 0.7714914083480835),
 ('shortfilm', 0.7663951516151428),
 ('nonmainstream', 0.7499780058860779),
 ('cassavettess', 0.7464396953582764),
 ('witnessedbr', 0.7415550351142883),
 ('directtodvd', 0.7393913865089417),
 ('criticsbr', 0.7382307648658752),
 ('fffc', 0.7371219396591187),
 ('tastesbr', 0.7359960079193115)]

'film' and 'movi' (movie) have similar vectors indicating this embedding seems to have worked

In [31]:
def transform_review_to_vector(review, model): #Create 1 vector of the text based on the Word2Vec vectors of the words in the sentence
    word_vectors = []
    for word in review:
        if word in model.wv:
            word_vectors.append(model.wv[word])
    if len(word_vectors) == 0:  # Handle case where no words are in vocabulary
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

Developping models: Embeddings, Features, Embeddings + Features. The architecture chosen starts at a high number of neurons, and then goes down gradually over the layers, so that it extracts more and more high level features. The data is balanced (same ratio for positives and negatives), so we use accuracy for model selection. The number of neurons for the model with only features was reduced (8 -> 4 -> 2)

In [33]:
X_train.shape

(32000, 7)

In [34]:
def prepare_data(choice, data, cols, model):
    if choice["Embedding"] == 1:
      data["Embedded Data"] = data["Stem Words"].apply(lambda x : transform_review_to_vector(x, model))
      df_embeddings = pd.DataFrame(data['Embedded Data'].tolist(), index=data.index)
    else:
      df_embeddings = pd.DataFrame()

    if choice["Features"] == 1:
      df_features = data[cols]
    else:
      df_features = pd.DataFrame()

    final_df = pd.concat([df_embeddings, df_features], axis=1)
    return final_df.to_numpy()
#Feedforward Neural Network (FNN)
choice_params = [{"Embedding" : 1, "Features" : 0}, {"Embedding" : 0, "Features" : 1}, {"Embedding" : 1, "Features" : 1}]
network_sizes = {(0, 1): (8, 4, 2)}
results = {}
for option in choice_params:
  X_train_opt = prepare_data(option, X_train, cols, model)
  X_val_opt = prepare_data(option, X_val, cols, model)
  X_test_opt = prepare_data(option, X_test, cols, model)


  # Input size depends on your feature representation (e.g., TF-IDF size or embedding size)
  input_dim = X_train_opt.shape[1]
  tuple_option = (option["Embedding"], option["Features"])
  if tuple_option in network_sizes.keys():
      # Define the FNN
    fnn = Sequential([
      Dense(network_sizes[tuple_option][0], activation='relu', input_shape=(input_dim,)),  # Input layer
      Dropout(0.5),  # Dropout for regularization
      Dense(network_sizes[tuple_option][1], activation='relu'),  # Hidden layer
      Dropout(0.4),
      Dense(network_sizes[tuple_option][2], activation = 'relu'),
      Dropout(0.3),
      Dense(1, activation='sigmoid')  # Output layer for binary classification
      ])
  else:
    fnn = Sequential([
      Dense(128, activation='relu', input_shape=(input_dim,)),  # Input layer
      Dropout(0.5),  # Dropout for regularization
      Dense(64, activation='relu'),  # Hidden layer
      Dropout(0.4),
      Dense(32, activation = 'relu'),
      Dropout(0.3),
      Dense(1, activation='sigmoid')  # Output layer for binary classification
      ])



      # Compile the model
  fnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
  history = fnn.fit(X_train_opt, y_train, validation_data=(X_val_opt, y_val), batch_size = 64, epochs=50, callbacks=[early_stopping])
  last_val_accuracy = history.history['val_accuracy'][-1]
  results[tuple_option] = last_val_accuracy

C:\Users\kkasp\anaconda3\envs\Text_mining\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - accuracy: 0.6807 - loss: 0.5707 - val_accuracy: 0.8618 - val_loss: 0.3328
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8432 - loss: 0.3715 - val_accuracy: 0.8702 - val_loss: 0.3091
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.8557 - loss: 0.3526 - val_accuracy: 0.8709 - val_loss: 0.3077
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8522 - loss: 0.3537 - val_accuracy: 0.8687 - val_loss: 0.3118
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.8572 - loss: 0.3480 - val_accuracy: 0.8739 - val_loss: 0.3046
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8582 - loss: 0.3401 - val_accuracy: 0.8726 - val_loss: 0.3032
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.8639 - loss: 0.3314 - val_accuracy: 0.8708 - val_loss: 0.3065
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.8624 - loss: 0.3311 - val_accu

C:\Users\kkasp\anaconda3\envs\Text_mining\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.5200 - loss: 0.7260 - val_accuracy: 0.6221 - val_loss: 0.6835
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.5369 - loss: 0.6792 - val_accuracy: 0.6816 - val_loss: 0.6509
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.5623 - loss: 0.6618 - val_accuracy: 0.7305 - val_loss: 0.6110
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.5924 - loss: 0.6547 - val_accuracy: 0.7354 - val_loss: 0.5938
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.6075 - loss: 0.6446 - val_accuracy: 0.7340 - val_loss: 0.5858
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.6210 - loss: 0.6345 - val_accuracy: 0.7355 - val_loss: 0.5719
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.6268 - loss: 0.6267 - val_accuracy: 0.7368 - val_loss: 0.5673
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.6350 - loss: 0.6222 - val_accurac

C:\Users\kkasp\anaconda3\envs\Text_mining\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.7178 - loss: 0.5448 - val_accuracy: 0.8509 - val_loss: 0.3520
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8334 - loss: 0.3874 - val_accuracy: 0.8540 - val_loss: 0.3420
Epoch 3/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.8503 - loss: 0.3569 - val_accuracy: 0.8687 - val_loss: 0.3126
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.8524 - loss: 0.3517 - val_accuracy: 0.8726 - val_loss: 0.3099
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.8582 - loss: 0.3437 - val_accuracy: 0.8706 - val_loss: 0.3024
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.8615 - loss: 0.3291 - val_accuracy: 0.8734 - val_loss: 0.3060
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - accuracy: 0.8622 - loss: 0.3301 - val_accuracy: 0.8770 - val_loss: 0.3011
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.8668 - loss: 0.3261 - val_accur

In [35]:
results

{(1, 0): 0.8712499737739563,
 (0, 1): 0.737375020980835,
 (1, 1): 0.8803750276565552}

(1, 0): The model is trained using only embeddings and achieves an accuracy of approximately 87.12%
(0, 1): The model uses only the predefined feature columns and achieves an accuracy of approximately 73.74%
(1, 1): The model combines both embeddings and feature columns and achieves an accuracy of approximately 88.04%

In [36]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [40]:
y_pred_probs = fnn.predict(X_test_opt)  # Predicted probabilities
y_pred = (y_pred_probs > 0.5).astype(int)  # Convert probabilities to binary labels (0 or 1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print evaluation metrics
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Precision: 0.8636009353078722
Recall: 0.8795395911887279
F1 Score: 0.8714973945531412


We achieve a precision of 86.36%, a recall score of 87.95% and an F1 score of 87.15%. These high scores indicate our model is quite strong in predicting the sentiment of reviews based on the review text.